In [23]:
import numpy as np
import pandas as pd
import utils

In [38]:
dtypes = {
    'Unnamed': "Int64",
    'Datetime': object,
    'Tweet Id':"Int64",
    'Raw content': str,
    'Text': str,
    'Username': str,
    'Reply Count': "Int16",
    'Retweet Count': "Int16",
    'Like Count': "Int16",
    'Quote Count': "Int16",
    'View Count': "Int16",
    'Hashtags': object,
    'Media': object,
    'Coordinates': object,
    'Place': object

}
kaggle = pd.read_csv("Kaggle_dataset.csv", low_memory = False)

In [52]:
kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26521 entries, 0 to 26520
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     26521 non-null  object 
 1   Datetime       26521 non-null  object 
 2   Tweet Id       26520 non-null  float64
 3   Raw content    26520 non-null  object 
 4   Text           26519 non-null  object 
 5   Username       26519 non-null  object 
 6   Reply Count    26518 non-null  float64
 7   Retweet Count  26518 non-null  float64
 8   Like Count     26518 non-null  float64
 9   Quote Count    26518 non-null  float64
 10  View Count     0 non-null      float64
 11  Hashtags       7154 non-null   object 
 12  Media          6991 non-null   object 
 13  Coordinates    557 non-null    object 
 14  Place          548 non-null    object 
 15  Captions       26521 non-null  object 
 16  Ext Text       26521 non-null  object 
dtypes: float64(6), object(11)
memory usage: 3.4+ MB


In [53]:
kaggle["Captions"] = np.empty((len(kaggle), 0)).tolist()

In [54]:
kaggle["Ext Text"] = np.empty((len(kaggle), 0)).tolist()

In [55]:
sentences = utils.get_loaded_json_file('kaggle_BILP2-generated-sentences.json')

In [56]:
print(len(sentences))

6563


In [57]:
kaggle['Media'] = kaggle['Media'].apply(lambda x: utils.fix_media_field(x))

In [58]:
type(list(kaggle['Media'])[0])

float

In [66]:
for index, row in kaggle.iterrows():
    if isinstance(row['Media'], float):
         kaggle.at[index, 'Ext Text'] = row['Text']
         continue
    media_list = row['Media']
    captions = []
    for m in media_list:
        try:
            filename = utils.get_filename_from_twitter_media_url(m)
        except:
            filename = utils.get_filename_format_from_url(m)

        if filename in sentences:
            caption = sentences[filename]

        if caption not in captions:
            captions.append(caption)

    kaggle.at[index, 'Captions'] = captions

    ext_text = row['Text']

    if len(captions) == 1:
        ext_text += " " + captions[0]

    elif len(captions) > 1:
        ext_text = "The images attached to the tweet depict "

        for caption in captions:
            c = caption.split("The image attached to the tweet depicts")[1]
            ext_text = ext_text + c + ","

        ext_text = row['Text'] + " " + ext_text

    kaggle.at[index, 'Ext Text'] = ext_text


In [67]:
kaggle['Ext Text'][9]

'Tropical Storm Harvey Storm Surge Watch/Warning Map dlvr.it/Phvnq8'

In [68]:
kaggle.to_csv("Kagglel_dataset_captions.csv")